#Regra do trapézio - OMP CRITICAL

##evitando a situação de corrida

In [43]:
%%writefile trap.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void Trap(double a,double b,int n,double *global_result_p);
double f(double x);

int main(int argc,char *argv[]){
	double global_result = 0.0;
	double a = 0;
	double b = 1;
	int n = 100;
	int thread_count;

	thread_count = strtol(argv[1], NULL, 10);
	#  pragma omp parallel num_threads(thread_count)
	Trap(a, b, n, &global_result);

	printf("With n = %d trapezoids, our estimate\n", n);
	printf("of the integral from %f to %f = %.14e\n", a, b, global_result);
	return 0;
}

double f(double x) {
	return x*x;
}

void Trap(double a,double b,int n,double *global_result_p){
	double h, x, myresult;
	double locala, localb;
	int i, localn;
	int myrank = omp_get_thread_num();
	int thread_count = omp_get_num_threads();

	h = (b-a)/n;
	localn = n/thread_count;
	locala = a + myrank*localn*h;
	localb = locala + localn*h;
	myresult = (f(locala) + f(localb))/2.0;

	for(i = 1; i <= localn-1; i++){
		x = locala + i*h;
		myresult += f(x);
	}

	myresult = myresult*h;

	#  pragma omp critical
	*global_result_p += myresult;
}

Writing trap.c


In [44]:
!gcc -fopenmp trap.c -o trap

In [45]:
!./trap 10

With n = 100 trapezoids, our estimate
of the integral from 0.000000 to 1.000000 = 3.33350000000000e-01


#pragma parellel for

##divisão de Trabalhao entre as Threads

In [12]:
%%writefile parallelfor.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void Trap(double a,double b,int n,double *global_result_p);
double f(double x);

int main(int argc,char *argv[]){
	double global_result = 0.0;
	double a = 0;
	double b = 1;
	int n = 100;
	int thread_count;

	thread_count = strtol(argv[1], NULL, 10);
	#  pragma omp parallel num_threads(thread_count)
	Trap(a, b, n, &global_result);

	printf("With n = %d trapezoids, our estimate\n", n);
	printf("of the integral from %f to %f = %.14e\n", a, b, global_result);
	return 0;
}

double f(double x) {
	return x*x;
}

void Trap(double a,double b,int n,double *global_result_p){
	double h, x, approx;
	double locala, localb;
	int i, localn;
	int myrank = omp_get_thread_num();
	int thread_count = omp_get_num_threads();

	h = (b-a)/n;
	localn = n/thread_count;
	locala = a + myrank*localn*h;
	localb = locala + localn*h;
  approx = (f(a) + f(b))/2.0;

  #  pragma omp parallel for num_threads(thread_count) \
    reduction(+: approx)
  for(i = 1; i <= n-1; i++)
    approx += f(a + i*h);
  approx = h*approx;

  #  pragma omp critical
	*global_result_p += approx;
}

Overwriting parallelfor.c


In [13]:
!gcc -fopenmp parallelfor.c -o parallelfor

In [14]:
!./parallelfor 10

With n = 100 trapezoids, our estimate
of the integral from 0.000000 to 1.000000 = 3.33350000000000e+00


#Usado errado parallel for, sequencia de fibonacci

In [23]:
%%writefile fibonacci.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc,char *argv[]){
  int n = 10;
  int fibo[n];
  int thread_count;

	thread_count = strtol(argv[1], NULL, 10);

  fibo[0] = fibo[1] = 1;
  # pragma omp parallel for num_threads(thread_count)
    for (int i = 2; i < n; i++)
      fibo[i] = fibo[i-1] + fibo[i-2];

  for(int i=0; i<n; i++) {
    printf("%d ", fibo[i]);
  }
}

Overwriting fibonacci.c


In [24]:
!gcc -fopenmp fibonacci.c -o fibonacci

In [26]:
!./fibonacci 10

1 1 2 3 2 1070786172 1070817415 2141603587 1070848658 1070879901 

#Cálculo de $\pi$ no parallel for definindo o escopo de cada variável

In [8]:
%%writefile pifor.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc,char *argv[]){
  int n = 1000;
  float factor = 1.0;
  int k;
  double sum = 0.0;

  int thread_count;
	thread_count = strtol(argv[1], NULL, 10);

  # pragma omp parallel for num_threads(thread_count) \
    default(none) reduction(+:sum) private(k, factor) \
    shared(n)
  for(k = 0; k < n; k++){
    if(k % 2 == 0)
        factor = 1.0;
    else
        factor = -1.0;
    sum += factor/(2*k+1);
  }

  printf("Resultado: %f", 4*sum);

}

Overwriting pifor.c


In [9]:
!gcc -fopenmp pifor.c -o pifor

In [11]:
!./pifor 100

Resultado: 3.140593